In [79]:
import wave
#import pydub
import numpy as np
import pandas as pd
import librosa
import os
import matplotlib.pyplot as plt
#import keras
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [80]:
def splitting(name):
    return [os.path.splitext(name)[1]],  os.path.splitext(name)[0].split(';')[:-1]

In [81]:
#Каталог из которого будем брать файлы
directory = './Voices/'
#Получаем список файлов в переменную files
files = os.listdir(directory) 

In [235]:
#cut the middle values and make 10 audios from 1
lst = []
audio = []
min_len = 110584
constant = min_len // 20
for i in range(len(files)):
  #  print(files[i])
    if files[i][0]=='1' or files[i][0]=='2':
        y, sr = librosa.load(directory+files[i])
        extension, name = splitting(files[i])
        interval = (len(y) - min_len) // 2
        for j in range(20):
            
            temp = []
            for k in range(constant):
                temp.append(y[interval+constant*j+k])
#                 lst.append(name+extension+[y[interval+constant*j+k]]+[sr])
#                 print(lst)
            audio.append(np.array(temp))
            lst.append(name + extension + temp + [sr])
        print('{} out of {}'.format(i, len(files)))

0 out of 110
1 out of 110
2 out of 110
3 out of 110
4 out of 110
5 out of 110
6 out of 110
7 out of 110
8 out of 110
9 out of 110
10 out of 110
11 out of 110
12 out of 110
13 out of 110
14 out of 110
15 out of 110
16 out of 110
17 out of 110
18 out of 110
19 out of 110
20 out of 110
21 out of 110
22 out of 110
23 out of 110
24 out of 110
25 out of 110
26 out of 110
27 out of 110
28 out of 110
29 out of 110
30 out of 110
31 out of 110
32 out of 110
33 out of 110
34 out of 110
35 out of 110
36 out of 110
37 out of 110
38 out of 110
39 out of 110
40 out of 110
41 out of 110
42 out of 110
43 out of 110
44 out of 110
45 out of 110
46 out of 110
47 out of 110
48 out of 110
49 out of 110
50 out of 110
51 out of 110
52 out of 110
53 out of 110
54 out of 110
55 out of 110
56 out of 110
57 out of 110
58 out of 110
59 out of 110
60 out of 110
61 out of 110
62 out of 110
63 out of 110
64 out of 110
65 out of 110
66 out of 110
67 out of 110
68 out of 110
69 out of 110
70 out of 110
71 out of 110
72

In [236]:
audio = np.array(audio)
audio.shape

(2200, 5529)

In [237]:
audios = ['Audio_{}'.format(i) for i in range(constant)]
col_names = ['Num', 'Name', 'Age', 'Gender', 'Ext'] + audios + ['Sr']
data = pd.DataFrame(lst, columns=col_names)
data.Name = data.Name.astype('category')
data.Name = data.Name.cat.rename_categories([i for i in range(data.Name.nunique())])
data.head()

,Num,Name,Age,Gender,Ext,Audio_0,Audio_1,Audio_2,Audio_3,Audio_4,...,Audio_5520,Audio_5521,Audio_5522,Audio_5523,Audio_5524,Audio_5525,Audio_5526,Audio_5527,Audio_5528,Sr
0,2,3,23,F,.wav,0.037744,0.042963,0.045602,0.048476,0.049683,...,-0.003041,0.006708,0.017953,0.017462,-0.002477,-0.020478,-0.015324,-0.008388,-0.000291,22050
1,2,3,23,F,.wav,0.009350,0.015799,0.005245,-0.014828,-0.019583,...,0.000911,0.000557,0.001369,0.001584,0.001286,0.000359,-0.000251,0.001169,-0.000582,22050
2,2,3,23,F,.wav,0.001462,0.002927,0.001584,0.001837,0.003543,...,0.000050,0.000428,0.000715,0.001001,0.000551,0.000111,-0.000679,-0.000291,0.000033,22050
3,2,3,23,F,.wav,0.000653,0.000990,0.000502,0.000289,0.000176,...,0.041023,0.039918,0.035112,0.024913,0.015266,0.004326,-0.006412,-0.009296,-0.008877,22050
4,2,3,23,F,.wav,-0.014228,-0.018208,-0.013448,-0.011329,-0.014752,...,-0.016211,-0.014956,-0.012936,-0.010687,-0.012107,-0.011594,-0.011450,-0.011019,-0.009027,22050


In [306]:
len(data)

2200

In [307]:
data_hayam = data[data['Num'] != '2']
data_turtle = data[data['Num'] == '2']
data_turtle.head()

,Num,Name,Age,Gender,Ext,Audio_0,Audio_1,Audio_2,Audio_3,Audio_4,...,Audio_5520,Audio_5521,Audio_5522,Audio_5523,Audio_5524,Audio_5525,Audio_5526,Audio_5527,Audio_5528,Sr
0,2,3,23,F,.wav,0.037744,0.042963,0.045602,0.048476,0.049683,...,-0.003041,0.006708,0.017953,0.017462,-0.002477,-0.020478,-0.015324,-0.008388,-0.000291,22050
1,2,3,23,F,.wav,0.009350,0.015799,0.005245,-0.014828,-0.019583,...,0.000911,0.000557,0.001369,0.001584,0.001286,0.000359,-0.000251,0.001169,-0.000582,22050
2,2,3,23,F,.wav,0.001462,0.002927,0.001584,0.001837,0.003543,...,0.000050,0.000428,0.000715,0.001001,0.000551,0.000111,-0.000679,-0.000291,0.000033,22050
3,2,3,23,F,.wav,0.000653,0.000990,0.000502,0.000289,0.000176,...,0.041023,0.039918,0.035112,0.024913,0.015266,0.004326,-0.006412,-0.009296,-0.008877,22050
4,2,3,23,F,.wav,-0.014228,-0.018208,-0.013448,-0.011329,-0.014752,...,-0.016211,-0.014956,-0.012936,-0.010687,-0.012107,-0.011594,-0.011450,-0.011019,-0.009027,22050


In [308]:
train_ffts_hayam = []

for i in range(data_hayam.shape[0]):
    elem = data_hayam[audios].iloc[i].values
    d = np.abs(librosa.stft(elem, n_fft=1024))
    train_ffts_hayam.append(d)

In [310]:
np.array(train_ffts_hayam).shape

(1100, 513, 22)

In [383]:
#shape = (1100,513,44)
target = data.Name
target_hayam = data_hayam.Name
target_turtle = data_turtle.Name


In [ ]:
train_ffts_hayam = np.array(train_ffts_hayam)
train_ffts_hayam = train_ffts_hayam.reshape(data_hayam.shape[0],513,22,1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_ffts_hayam, target_hayam, test_size=0.1, shuffle=True)

In [384]:
X_train.shape

(990, 513, 22, 1)

In [385]:
import tensorflow as tf
import keras
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
from keras.layers import Dense, Dropout, Activation, Flatten

In [386]:

input_shape = (513,22,1)
model = keras.models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=input_shape))
model.add(Activation('tanh'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation=tf.nn.relu))
model.add(Dropout(0.25))
model.add(Dense(55,activation=tf.nn.softmax)) 

In [387]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [388]:
model.fit(x=X_train,y=y_train, epochs=20, batch_size=10) 

Epoch 1/20
990/990 [==============================] - 47s 47ms/step - loss: 3.9454 - acc: 0.0323
Epoch 2/20
990/990 [==============================] - 43s 43ms/step - loss: 3.5302 - acc: 0.1232
Epoch 3/20
990/990 [==============================] - 43s 43ms/step - loss: 2.7425 - acc: 0.2687
Epoch 4/20
990/990 [==============================] - 43s 43ms/step - loss: 1.9539 - acc: 0.4626
Epoch 5/20
990/990 [==============================] - 43s 43ms/step - loss: 1.3135 - acc: 0.6253
Epoch 6/20
990/990 [==============================] - 43s 44ms/step - loss: 0.8854 - acc: 0.7455
Epoch 7/20
990/990 [==============================] - 43s 43ms/step - loss: 0.5572 - acc: 0.8394
Epoch 8/20
990/990 [==============================] - 43s 43ms/step - loss: 0.5316 - acc: 0.8545
Epoch 9/20
990/990 [==============================] - 43s 43ms/step - loss: 0.3649 - acc: 0.8838
Epoch 10/20
990/990 [==============================] - 44s 44ms/step - loss: 0.2743 - acc: 0.9172
Epoch 11/20
990/990 [========

KeyboardInterrupt: 

In [389]:
pred = model.predict(X_test)
prediction = []
for string in pred:
    a = np.argmax(string)
    prediction.append(a)
#print(prediction)

In [390]:
#print(y_test)

In [391]:
#loss with fft
print(accuracy_score(prediction, y_test))

0.4818181818181818


In [238]:
list_of_mfccs = []
for i in range(len(data)):
    mfcc = librosa.feature.mfcc(data[audios].iloc[i].values, data.Sr.iloc[i])
    mfcc = np.array(mfcc)
    list_of_mfccs.append(mfcc)
    if i % 100 == 0:
        print('{} out of {} (1)'.format(i, len(data)))
    
#list_of_mfccs = np.array(list_of_mfccs)
#mfcc_names = ['mfcc_{}'.format(i) for i in range(list_of_mfccs.shape[1])]
#for i in range(list_of_mfccs.shape[1]):
#    data['mfcc_{}'.format(i)] = list_of_mfccs[:, i]

0 out of 2200 (1)
100 out of 2200 (1)
200 out of 2200 (1)
300 out of 2200 (1)
400 out of 2200 (1)
500 out of 2200 (1)
600 out of 2200 (1)
700 out of 2200 (1)
800 out of 2200 (1)
900 out of 2200 (1)
1000 out of 2200 (1)
1100 out of 2200 (1)
1200 out of 2200 (1)
1300 out of 2200 (1)
1400 out of 2200 (1)
1500 out of 2200 (1)
1600 out of 2200 (1)
1700 out of 2200 (1)
1800 out of 2200 (1)
1900 out of 2200 (1)
2000 out of 2200 (1)
2100 out of 2200 (1)


In [239]:
train_mfccs = np.array(list_of_mfccs)
train_mfccs.shape

(2200, 20, 11)

In [392]:
train_mfccs = train_mfccs.reshape(2200,20,11,1)
X_train, X_test, y_train, y_test = train_test_split(train_mfccs, target, test_size=0.1, shuffle=True)

In [431]:
input_shape = (20,11,1)
model = keras.models.Sequential()
model.add(Conv2D(16, (2, 2), padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
#model.add(Dropout(0.25))
#model.add(Conv2D(32, (2, 2)))
#model.add(Activation('relu'))
#model.add(Conv2D(32, (2, 2)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(256, activation=tf.nn.relu))
#model.add(Dropout(0.5))
model.add(Dense(55,activation=tf.nn.softmax)) 

In [432]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [433]:
model.fit(x=X_train,y=y_train, epochs=20, batch_size=20)

Epoch 1/20
1980/1980 [==============================] - 4s 2ms/step - loss: 7.7776 - acc: 0.0611
Epoch 2/20
1980/1980 [==============================] - 1s 324us/step - loss: 2.4604 - acc: 0.3389
Epoch 3/20
1980/1980 [==============================] - 1s 325us/step - loss: 1.3619 - acc: 0.6152
Epoch 4/20
1980/1980 [==============================] - 1s 325us/step - loss: 0.9005 - acc: 0.7359
Epoch 5/20
1980/1980 [==============================] - 1s 325us/step - loss: 0.6058 - acc: 0.8111
Epoch 6/20
1980/1980 [==============================] - 1s 326us/step - loss: 0.4263 - acc: 0.8677
Epoch 7/20
1980/1980 [==============================] - 1s 324us/step - loss: 0.3391 - acc: 0.8970
Epoch 8/20
1980/1980 [==============================] - 1s 329us/step - loss: 0.2523 - acc: 0.9288
Epoch 9/20
1980/1980 [==============================] - 1s 327us/step - loss: 0.1889 - acc: 0.9485
Epoch 10/20
1980/1980 [==============================] - 1s 324us/step - loss: 0.1499 - acc: 0.9601
Epoch 11/20

In [434]:
#loss with mfccs
pred = model.predict(X_test)
prediction = []
for string in pred:
    a = np.argmax(string)
    prediction.append(a)
print(accuracy_score(prediction, y_test))

0.6090909090909091
